# Hello class!


Welcome to my individual project! 

### What is this thing?

This is an interactive "Flux based model" of ecoli metabolism. It is created using the [python](http://www.python.org) programming language running in a [jupyter notebook](http://jupyter.org/). 

### What is a Flux Based Model?

Organisms convert energy and food into useful molecules and proginy through series of many individual reactions. Estimating the kinetic parameters for each of these reactions under the varying conditions within in a cell is often impractical. Flux based models use stoicheometry of the reactions and some complicated math (linear optimization) to predict the concentrations of metabolites and the "flux" through reactions.

### How does it work

1. Input all of the known reactions of an organism into the model in terms of their stoicheometry.
2. Define an "objective function", for example cell growth rate.
3. Using linear optimization to see what reactions would be employed to maximize the objective function

## An example to give you a feel for the technique

One pathway in ecoli metabolism is the creation of the amino acid threonine.


In [1]:
import numpy as np

## References


1. W. Covert. Fundamentals of Systems Biology: From Synthetic Circuits to Whole-cell Models Markus. CRC Press. 2015.
2. Keseler et al. (2017), "EcoCyc: reflecting new knowledge about Escherichia coli K-12", Nucleic Acids Research 45:D543-50.

In [4]:
import requests
r = requests.get('https://websvc.biocyc.org/apixml?fn=genes-of-pathway&id=ECOLI:HOMOSER-THRESYN-PWY&detail=full')

In [3]:
from lxml.html import Element, fromstring
fromstring(r.text)

ModuleNotFoundError: No module named 'lxml'

In [29]:
from xmljson import badgerfish 
from xml.etree.ElementTree import fromstring
import requests

r = requests.get('https://websvc.biocyc.org/apixml?fn=genes-of-pathway&id=ECOLI:HOMOSER-THRESYN-PWY&detail=full')
res = fromstring(r.text)

In [36]:
genes = res.findall('Gene')
g = genes[0]


In [163]:
class BioCycAPI(object):
    def __init__(self):
        self.cache = {}

    def get_xml(self, identifier):
        if identifier in self.cache:
            return self.cache[identifier]
        
        resp = requests.get('https://websvc.biocyc.org/getxml?%s' % identifier)
        if resp.status_code != 200:
            raise Exception('Error fetching biocyc identifier "%s"' % identifier)
        result = fromstring(resp.text)
        self.cache[identifier] = result
        return result
        
    def get_compound(self, compound_identifier):        
        result = self.get_xml(compound_identifier)
        compound_xml = result.find('Compound')

        return {
            'common_name': compound_xml.find('common-name').text,
            'synonym': compound_xml.find('synonym').text,
            #'inchi': compound_xml.find('inchi').text
        }

    def parse_reaction_participant_xml(self, participant_xml):
        compound = participant_xml.find('Compound')
        compound_identifier = compound.attrib['resource'][7:]

        coefficient_xml = participant_xml.find('coefficient')
        if coefficient_xml is None:
            coefficient = 1
        else:
            coefficient = int(coefficient_xml.text)

        participant_details = api.get_compound(compound_identifier)
        participant_details['coefficient'] = coefficient
        return participant_details
    
    def get_reaction(self, reaction_identifier):
        result = self.get_xml(reaction_identifier)
        rxn_xml = result.findall('Reaction')[0]

        er_xml = rxn_xml.find('enzymatic-reaction')
        er_xml = er_xml.find('Enzymatic-Reaction')
        
        rxn_details = {
            'enzyme_ec_number': rxn_xml.find('ec-number').text,
            'enzyme_common_name': er_xml.find('common-name').text,
            'enzyme_synonym': er_xml.find('synonym').text
        }
        
        reactants = []
        for reactant_xml in rxn_xml.findall('left'):
            reactants.append(self.parse_reaction_participant_xml(reactant_xml))
        products = []
        for product_xml in rxn_xml.findall('right'):
            products.append(self.parse_reaction_participant_xml(product_xml))
        
        rxn_details['reactants'] = reactants
        rxn_details['products'] = products
        
        return rxn_details

api = BioCycAPI()
x = api.get_reaction('ECOLI:THRESYN-RXN')
import pprint
pprint.pprint(x)
#api.get_compound('ECOLI:O-PHOSPHO-L-HOMOSERINE')

{'enzyme_common_name': 'threonine synthase',
 'enzyme_ec_number': 'EC-4.2.3.1',
 'enzyme_synonym': 'O-phospho-L-homoserine phospho-lyase',
 'products': [{'coefficient': 1,
               'common_name': 'phosphate',
               'synonym': 'inorganic phosphate'},
              {'coefficient': 1, 'common_name': 'L-threonine', 'synonym': 'T'}],
 'reactants': [{'coefficient': 1,
                'common_name': 'O-phospho-L-homoserine',
                'synonym': 'o-phosphohomoserine'},
               {'coefficient': 1,
                'common_name': 'H<sub>2</sub>O',
                'synonym': 'H2O'}]}


In [154]:
rxn_xml = xml.findall('Reaction')[0]
reactants = []
for reactants_xml in rxn_xml.findall('left'):
    compound = reactants_xml.find('Compound')
    compound_identifier = compound.attrib['resource'][7:]
    
    coefficient_xml = product.find('coefficient')
    if coefficient_xml is None:
        coefficient = 1
    else:
        coefficient = int(coefficient_xml.text)
    
    reactant_details = api.get_compound(compound_identifier)
    reactant_details['coefficient'] = coefficient
    
print(reactants)

[]


In [88]:
er_xml = rxn_xml.find('enzymatic-reaction')
er_xml = er_xml.find('Enzymatic-Reaction')
er_xml.find('common-name').text
er_xml.find('synonym').text

'O-phospho-L-homoserine phospho-lyase'

In [158]:
x

{'enzyme_ec_number': 'EC-4.2.3.1',
 'enzyme_common_name': 'threonine synthase',
 'enzyme_synonym': 'O-phospho-L-homoserine phospho-lyase',
 'reactants': [{'common_name': 'O-phospho-L-homoserine',
   'synonym': 'o-phosphohomoserine',
   'coefficient': 2},
  {'common_name': 'H<sub>2</sub>O', 'synonym': 'H2O', 'coefficient': 2}]}

In [141]:
res = api.get_xml('ECOLI:RXN0-5268')

In [153]:
for product in res.find('Reaction').findall('left'):
    coefficient_xml = product.find('coefficient')
    if coefficient_xml is None:
        coefficient = 1
    else:
        coefficient = int(coefficient_xml.text)
    
    print(coefficient)

8
1
2
